### Machine Learning based Performance modelling

> Performance modelling revolves around predicting the **execution times** of an **algorithm**.

### Defining Algorithm and Kernel

An **Algorithm** is usually a psuedo code of a computer program which will compute certain mathematical equation. Suppose you have a matrix chain (AB)C, to compute this equation, it has to be decomposed into certain standard operations.

```c++
ABC(){
    // (AB)C
    T = AB
    D = TC     
}
```

Usually there are specilized computer porgrams that compute these standard operations and they are called **kernels**. 

```c++
void Algorithm(){
    // (AB)C
    dgemm(A,B,200x200,200x500,T) 
    dgemm(T,C,200x500,500x100,D) 
} 
```

So from our perspective, **an algorithm is a sequence of kernel calls**

### Non deterministic nature of execution times

The execution time is non deterministic because of certain factors like CPU temperature, data locality etc. So when an algorithm is repeated many times, certain distribution is observed. 

The statistical dispersion of the execution time depends on the amount of work needed for that computation. The following graph shows measurements of matrix multiplication for different matrix sizes. You can see that when the matrix size is small the dispersion is larger. The color dots are the 15th (red) and 85th (green) quantile - that is I repeat each matrix multiplation 100 times, sort them according to execution time and these dots are 15th and 85th element in that sort.

<img src="pics/quantile_reg.png" />

### Research : Performance modelling

So we think it's better if instead of summarizing execution time as a single number like mean or median, we ll provide an interval, ie an estimate which says execution time will lie between certain range for, say 70% of the time

So we want to fit quantiles or use a quantile loss which assigns different penalities for over and under estimation. 

$L_q(y, \mathbf{f}) = \sum_{i=y_i < \mathbf{f}_i} (1-q)|y_i - \mathbf{f}_i | + \sum_{i=y_i \ge \mathbf{f}_i} (q)|y_i - \mathbf{f}_i | $

<img src="pics/quantile_loss.png" />



The scatter plot shown in the first figure is for a single kernel, but in reality an alogirthm is composed of a sequence of kernel calls and we have to add up estimates of individual kernels. In most cases simple adding might work, but it does not take into effect caching and there might be certain underestimation because of this.

```c++
void Algorithm(){
    // (AB)C
    dgemm(A,B,200x200,200x500,B) ---> Qa_1,Qa_2,Qa_3
    dgemm(B,C,200x500,500x100,D) ---> Qb_1,Qb_2,Qb_3
} ---> Qfoo_1, Qfoo_2, Qfoo_3
```

### Research: Experiment design

The scatter plot shown in first figure is going to vary from machine to machine. It also needs to be trained separately for each kernel

Therefore, we want to do some experiment design - minimize the number of measurements and at the same time do not compramise on the quality of solutions. Especially the measurements towards the end of the graph takes more time.


### Feedback :

> Use the word "library calls" instead of "kernels"

> How much more information do the quantiles have more than summaries like min / median

> Other ways : Fitting distributions? (But how easy is it to design experiments then?

> Can we replace MLP with some other regression function? (right now the input is just some numbers, but what if we move towards sparse solutions when we want to have some categorical variables as input)

> Non linear solutions implies iterative solutions

> Predicting performance of iterative algorithm. (We also have to consider sparse computations)

> Armijo -- step size control

### Iterative algorithms

```c++
while rersidual < eps{
        ...
        Algorithm(...) // execution time might be different for different iterations
        ...
}

```

Note: **Time for each iteration can be different**:

> If the input for each iteration is **sparse**, the run time for each iteration might depend on the number of non zero elements in the input.  

> Sometimes, the algorithm itself might have code which increases computation or storage access as iteration proceeds. Eg, Arnoldi iterations of GMRES